In [9]:
import seaborn as sns
import operator as op
import matplotlib.pyplot as plt
import numpy as np ##for arrays
import pandas as pd ## for dataframe
import re ##searching words
from nltk.corpus import stopwords## words that dont add value like "the,a,were"
from nltk.stem.porter import PorterStemmer##give rootwords of a particular word
from sklearn.feature_extraction.text import TfidfVectorizer ##convert text into feature vectors
from sklearn.model_selection import train_test_split ## splitting data into train,test
from sklearn.linear_model import LogisticRegression ## ML model
from sklearn.metrics import accuracy_score## accuracy

In [10]:
# pip install scikit-learn

In [11]:
import nltk
nltk.download('stopwords') # downloading all the stopwords if not already present

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
print(stopwords.words('english')) # printing the stopwords in English

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
# loading the dataset to a pandas DataFrame
train = pd.read_csv('C:/Fleetronix/ML/Batch 2/Fake-News-Classification-using-LSTM-master/Fake-News-Classification-using-LSTM-master/train.csv')
test = pd.read_csv('C:/Fleetronix/ML/Batch 2/Fake-News-Classification-using-LSTM-master/Fake-News-Classification-using-LSTM-master/test.csv')

In [14]:
# print the first 5 rows of the dataframe
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [16]:
# counting the number of missing values in the dataset
print(train.isnull().sum())
print(test.isnull().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
id          0
title     122
author    503
text        7
dtype: int64


In [17]:
# replacing the null values with empty string
train = train.fillna('')
test = test.fillna('')

In [18]:
X = train['author'] + ' ' + train['title']
X_test = test['author'] + ' ' + test['title']

In [19]:
print(X.head())
print(X_test.head())

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
dtype: object
0    David Streitfeld Specter of Trump Loosens Tong...
1     Russian warships ready to strike terrorists n...
2    Common Dreams #NoDAPL: Native American Leaders...
3    Daniel Victor Tim Tebow Will Attempt Another C...
4    Truth Broadcast Network Keiser Report: Meme Wa...
dtype: object


In [20]:
Y = train['label'] # storing label in Y

In [21]:
print(Y.head())

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64


In [22]:
port_stem = PorterStemmer() # creating instance
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # replace anything other than a_z with ' '
    stemmed_content = stemmed_content.lower() # convert to lower case 
    stemmed_content = stemmed_content.split() # split all the words to list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # stemming + removing stopwords
    stemmed_content = ' '.join(stemmed_content) # again joining all words with ' '
    return stemmed_content

X = X.apply(stemming)
X_test = X_test.apply(stemming)

In [23]:
print(X.head(), X_test.head())

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
dtype: object 0    david streitfeld specter trump loosen tongu pu...
1    russian warship readi strike terrorist near al...
2    common dream nodapl nativ american leader vow ...
3    daniel victor tim tebow attempt anoth comeback...
4     truth broadcast network keiser report meme war e
dtype: object


In [24]:
X = X.values # Data Array
Y = Y.values # Label
X_test = X_test.values

In [25]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [26]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(X)
X_test = vectorizer.transform(X_test)

In [27]:
print(X)

  (np.int32(0), np.int32(3600))	0.3598939188262559
  (np.int32(0), np.int32(8909))	0.3635963806326075
  (np.int32(0), np.int32(7005))	0.21874169089359144
  (np.int32(0), np.int32(3792))	0.2705332480845492
  (np.int32(0), np.int32(267))	0.27010124977708766
  (np.int32(0), np.int32(4973))	0.233316966909351
  (np.int32(0), np.int32(13473))	0.2565896679337957
  (np.int32(0), np.int32(2959))	0.2468450128533713
  (np.int32(0), np.int32(8630))	0.29212514087043684
  (np.int32(0), np.int32(7692))	0.24785219520671603
  (np.int32(0), np.int32(2483))	0.3676519686797209
  (np.int32(0), np.int32(15686))	0.28485063562728646
  (np.int32(1), np.int32(3568))	0.26373768806048464
  (np.int32(1), np.int32(5503))	0.7143299355715573
  (np.int32(1), np.int32(6816))	0.1904660198296849
  (np.int32(1), np.int32(2813))	0.19094574062359204
  (np.int32(1), np.int32(1497))	0.2939891562094648
  (np.int32(1), np.int32(16799))	0.30071745655510157
  (np.int32(1), np.int32(2223))	0.3827320386859759
  (np.int32(1), np.int

In [28]:
X_train, X_val, Y_train , Y_val = train_test_split(X, Y, test_size = 0.2, random_state=2)

In [29]:
clf = LogisticRegression()
clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_val)
Y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [30]:
accuracy = accuracy_score(Y_val, Y_pred)
print(f'Validation Accuracy: {accuracy}')

Validation Accuracy: 0.9774038461538461


In [31]:
X_pred = clf.predict(X_test)
output = pd.DataFrame({'id': test.id, 'label': X_pred})
output.to_csv('submission.csv', index=False)